In [18]:
from logic import *

class ALU():
    def __init__(self, name, zeroFlag = 1, overflowFlag = 0, negativeFlag = 0):
        self.name = name
        self.zeroFlag = zeroFlag
        self.overflowFlag  = overflowFlag
        self.negativeFlag= negativeFlag
        
    @staticmethod
    def toString(byte):
        s = ""
        for bit in byte:
            x = str(int(bit))
            s += x
        return s

    
    def isZero(self, byte):
        Out = Not("Out")
        def isSimpleZero(byte):
            if len(byte) == 2:
                F0 = Or("F0")
                F0.A.set(bit(byte, 0))
                F0.B.set(bit(byte, 1))
                return F0.C.value
            else:
                F1 = Or("F1")
                left = isSimpleZero(byte[0:int(len(byte)/2)])
                right = isSimpleZero(byte[int(len(byte)/2):int(len(byte))])
                F1.A.set(left)
                F1.B.set(right)
                # print(left, right)
                return (F1.C.value)
        Out.A.set(isSimpleZero(byte))
        Out.evaluate()
        flag = str(int(Out.B.value))
        self.zeroFlag = flag
        if(flag == "1"):
            print("Is Zero: ", self.zeroFlag)
        return flag
    
    def isNegative(self, byte):
        F0 = And("F0")
        F0.A.set(bit(byte, 0))
        F0.B.set(True)
        flag = str(int(F0.C.value))
        self.negativeFlag = flag
        if(flag == "1"):
            print("Is Negative: ", self.zeroFlag)
        return(flag)

    def addCarry(self, a, b):  
        F0 = FullAdder("F0")
        F1 = FullAdder("F1")
        F0.Cout.connect(F1.Cin)
        F2 = FullAdder("F2")
        F1.Cout.connect(F2.Cin)
        F3 = FullAdder("F3")
        F2.Cout.connect(F3.Cin)
        F4 = FullAdder("F4")
        F3.Cout.connect(F4.Cin)
        F5 = FullAdder("F5")
        F4.Cout.connect(F5.Cin)
        F6 = FullAdder("F6")
        F5.Cout.connect(F6.Cin)
        F7 = FullAdder("F7")
        F6.Cout.connect(F7.Cin)
            
        F0.Cin.set(0)
        F0.A.set(bit(a, 7))
        F0.B.set(bit(b, 7)) 
        F1.A.set(bit(a, 6))
        F1.B.set(bit(b, 6))
        F2.A.set(bit(a, 5))
        F2.B.set(bit(b, 5))
        F3.A.set(bit(a, 4))
        F3.B.set(bit(b, 4))
        F4.A.set(bit(a, 3))
        F4.B.set(bit(b, 3))
        F5.A.set(bit(a, 2))
        F5.B.set(bit(b, 2))
        F6.A.set(bit(a, 1))
        F6.B.set(bit(b, 1))
        F7.A.set(bit(a, 0))
        F7.B.set(bit(b, 0))
        
        flag = str(int(F7.Cout.value))
        self.overflowFlag = flag
        if (flag == "1"):
            print("Is Overflow: " , self.overflowFlag )
        array = [F7.S.value,F6.S.value,F5.S.value,
                                    F4.S.value,F3.S.value,F2.S.value,F1.S.value,F0.S.value]
        byte = self.toString(array)
        self.isNegative(byte)
        self.isZero(byte)
        return byte
    
    def increment(self, a):
        return self.toString(self.addCarry(a, "00000001"))

    def complementOne(self, a):
        F0 = Not("F0")
        F1 = Not("F1")
        F3 = Not("F3")
        F2 = Not("F2")
        F4 = Not("F4")
        F5 = Not("F5")
        F6 = Not("F6")
        F7 = Not("F7")

        F0.A.set(bit(a,7))
        F0.evaluate()
        F1.A.set(bit(a,6))
        F1.evaluate()
        F2.A.set(bit(a,5))
        F2.evaluate()
        F3.A.set(bit(a,4))
        F3.evaluate()
        F4.A.set(bit(a,3))
        F4.evaluate()
        F5.A.set(bit(a,2))
        F5.evaluate()
        F6.A.set(bit(a,1))
        F6.evaluate()
        F7.A.set(bit(a,0))
        F7.evaluate()
        
        byte = [F7.B.value, F6.B.value,F5.B.value,
                                    F4.B.value,F3.B.value,F2.B.value,F1.B.value,F0.B.value]
        return self.toString(byte)

    def negate(self, a):
        complement = self.complementOne(a)
        result = self.increment(complement)
        return self.toString(result)
        
    def decrement(self, a):
        return self.toString(self.addCarry(a, "11111111"))
        
    def passTrough(self, a):
        inverted = self.complementOne(a)
        result = self.complementOne(inverted)
        return self.toString(result)

    def substraction (self, a, b):
        b = self.toString(self.negate(b))
        c = self.addCarry(a, b)
        return self.toString(c)

In [20]:
hola = ALU("hola")
# hola.increment("01101101")
hola.substraction('00000011' ,'00000011')

Is Negative:  1
Is Overflow:  1
Is Zero:  1


'00000000'

In [122]:
import unittest

In [ ]:
class TestIsZero(unittest.TestCase):
    test = ALU("isZero")
    def test1(self):
        self.assertEqual(self.test.isZero('11000000'),'0'), "Should be False"
    def test2(self):
        self.assertEqual(self.test.isZero('01000010'),'0'), "Should be False"
    def test3(self):
        self.assertEqual(self.test.isZero('11000010'), '0'), "Should be False"
    def test4(self):
        self.assertEqual(self.test.isZero('11111111'), '0'), "Should be False"
    def test5(self):
        self.assertEqual(self.test.isZero('00000000'),'1'), "Should be True"

unittest.main(argv=[''], verbosity=2, exit=False)

In [ ]:
class TestIsNegative(unittest.TestCase):
    test = ALU("isNegative")
    def test1(self):
        self.assertEqual(self.test.isNegative('11000000'),'1'), "Should be True"
    def test2(self):
        self.assertEqual(self.test.isNegative('01000010'),'0'), "Should be False"
    def test3(self):
        self.assertEqual(self.test.isNegative('11000010'), '1'), "Should be True"
    def test4(self):
        self.assertEqual(self.test.isNegative('11111111'), '1'), "Should be True"
    def test5(self):
        self.assertEqual(self.test.isNegative('00000000'),'0'), "Should be False"

unittest.main(argv=[''], verbosity=2, exit=False)

In [ ]:
class TestAddCarry(unittest.TestCase):
    test = ALU("TestAddCarry")
    def test1(self):
        self.assertEqual(self.test.addCarry('11000000','10101000'),"01101000"), "101101000"
    def test2(self):
        self.assertEqual(self.test.addCarry('11000010','00101010'),"11101100"), "011101100"
    def test3(self):
        self.assertEqual(self.test.addCarry('11000010','00101010'),'11101100'), "11101100"
    def test4(self):
        self.assertEqual(self.test.addCarry('11000010','00101010'),"11101100"), "11101100"
    def test5(self):
        self.assertEqual(self.test.addCarry('01100010','10101011'),"00001101"), "100001101"
        
unittest.main(argv=[''], verbosity=2, exit=False)